# 問題
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ。

In [1]:
import pandas as pd

df = pd.read_json("Jawiki Country.json.gz", lines=True)
uk_text = df.query('title=="イギリス"')["text"].values[0]

In [ ]:
import re

# Category: または カテゴリ: に対応し、カテゴリ名だけをキャプチャ
pattern = r'^\s*\[\[(?:Category|カテゴリ):\s*([^|\]]+?)\s*(?:\|[^]]*)?\]\]\s*$'
result = re.findall(pattern, uk_text, flags=re.MULTILINE)

print(result)

[]


In [12]:
# ?:なしだと、、、グループ化で表示される。つまり、?:をつけるとグループ化で表示されなくなる
pattern = r'^\s*\[\[(Category|カテゴリ):\s*([^|\]]+?)\s*(?:\|[^]]*)?\]\]\s*$'
result = re.findall(pattern, uk_text, flags=re.MULTILINE)

print(result)

[('Category', 'イギリス'), ('Category', 'イギリス連邦加盟国'), ('Category', '英連邦王国'), ('Category', 'G8加盟国'), ('Category', '欧州連合加盟国'), ('Category', '海洋国家'), ('Category', '現存する君主国'), ('Category', '島国'), ('Category', '1801年に成立した国家・領域')]


1.	^
行頭アンカー。「この行の先頭からマッチ開始」。re.MULTILINEで各行に適用。
	2.	\s*
行頭の空白（スペースやタブ）があってもOKにする緩衝材。
	3.	\[\[
文字通りの [[。角括弧は正規表現で特別記号なので \[ とエスケープ。
	4.	(?:Category|カテゴリ):
非キャプチャグループ。
英語名空間 Category: と日本語名空間 カテゴリ: のどちらでもOKにする。
（?: は「グループ化するが結果に残さない」= 戻り値に要らないから）
	5.	\s*
コロンの後に入るかもしれない空白を許容（[[Category:  英国]] など）。
	6.	([^|\]]+?)  ← ★ここがカテゴリ名だけ

	•	(...) はキャプチャグループ（ここが戻り値になる）。
	•	[^|\]]+ は否定文字クラス：「| と ] 以外の文字」を1文字以上。
→ つまり | または ]] が来る直前までをスパッと取る。
	•	+? は最短一致（この否定クラスでは実質的に + と同じ効果。過剰取得防止の保険）。

例
[[Category:英国]] → キャプチャ = 英国
[[Category:ヨーロッパの国|Europe]] → キャプチャ = ヨーロッパの国

	7.	\s*
カテゴリ名末尾の余分な空白を吸収。
	8.	(?:\|[^]]*)?
非キャプチャ。任意の後置部（あってもなくてもOK）。

	•	\| で文字通りの |。
	•	[^]]* は ] 以外を0回以上 → ]] が来るまで全部スキップ。
→ ソートキー/表示名（例：|*, |United Kingdom）を“読み飛ばす”役。

	9.	\]\]
文字通りの ]] でカテゴリ宣言を閉じる。
	10.	\s*$
行末の空白を許容しつつ、行末で終わることを要求。